This exercise will require you to pull some data from the Qunadl API. Qaundl is currently the most widely used aggregator of financial market data.

As a first step, you will need to register a free account on the http://www.quandl.com website.

After you register, you will be provided with a unique API key, that you should store:

In [69]:
# Store the API key as a string - according to PEP8, constants are always named in all upper case
API_KEY = 

Qaundl has a large number of data sources, but, unfortunately, most of them require a Premium subscription. Still, there are also a good number of free datasets.

For this mini project, we will focus on equities data from the Frankfurt Stock Exhange (FSE), which is available for free. We'll try and analyze the stock prices of a company called Carl Zeiss Meditec, which manufactures tools for eye examinations, as well as medical lasers for laser eye surgery: https://www.zeiss.com/meditec/int/home.html. The company is listed under the stock ticker AFX_X.

You can find the detailed Quandl API instructions here: https://docs.quandl.com/docs/time-series

While there is a dedicated Python package for connecting to the Quandl API, we would prefer that you use the *requests* package, which can be easily downloaded using *pip* or *conda*. You can find the documentation for the package here: http://docs.python-requests.org/en/master/ 

Finally, apart from the *requests* package, you are encouraged to not use any third party Python packages, such as *pandas*, and instead focus on what's available in the Python Standard Library (the *collections* module might come in handy: https://pymotw.com/3/collections/ ).
Also, since you won't have access to DataFrames, you are encouraged to us Python's native data structures - preferably dictionaries, though some questions can also be answered using lists.
You can read more on these data structures here: https://docs.python.org/3/tutorial/datastructures.html

Keep in mind that the JSON responses you will be getting from the API map almost one-to-one to Python's dictionaries. Unfortunately, they can be very nested, so make sure you read up on indexing dictionaries in the documentation provided above.

# 1 - Get data from quandl api for a single day

In [70]:
# First, import the relevant modules
import requests
import collections
from datetime import datetime
url='https://www.quandl.com/api/v3/datasets/FSE/AFX_X.json?&start_date=2017-10-10&end_date=2017-10-10api_key=API_KEY'

r = requests.get(url)
json_data = r.json()


In [71]:
json_data

{'dataset': {'id': 10095370,
  'dataset_code': 'AFX_X',
  'database_code': 'FSE',
  'name': 'Carl Zeiss Meditec (AFX_X)',
  'description': 'Stock Prices for Carl Zeiss Meditec (AFX) from the Frankfurt Stock Exchange.<br><br>Trading System: Xetra<br><br>ISIN: DE0005313704',
  'refreshed_at': '2019-05-10T22:30:38.681Z',
  'newest_available_date': '2019-05-10',
  'oldest_available_date': '2000-06-07',
  'column_names': ['Date',
   'Open',
   'High',
   'Low',
   'Close',
   'Change',
   'Traded Volume',
   'Turnover',
   'Last Price of the Day',
   'Daily Traded Units',
   'Daily Turnover'],
  'frequency': 'daily',
  'type': 'Time Series',
  'premium': False,
  'limit': None,
  'transform': None,
  'column_index': None,
  'start_date': '2017-10-10',
  'end_date': '2017-10-10',
  'data': [['2017-10-10',
    45.64,
    46.04,
    45.57,
    45.84,
    None,
    65860.0,
    3016658.0,
    None,
    None,
    None]],
  'collapse': None,
  'order': None,
  'database_id': 6129}}

These are your tasks for this mini project:

1. Collect data from the Franfurt Stock Exchange, for the ticker AFX_X, for the whole year 2017 (keep in mind that the date format is YYYY-MM-DD).
2. Convert the returned JSON object into a Python dictionary.
3. Calculate what the highest and lowest opening prices were for the stock in this period.
4. What was the largest change in any one day (based on High and Low price)?
5. What was the largest change between any two days (based on Closing Price)?
6. What was the average daily trading volume during this year?
7. (Optional) What was the median trading volume during this year. (Note: you may need to implement your own function for calculating the median.)

#  2 - Below we collect the data from API for the entire period 2017 using the start_date and end_date parameters, store the same in json format

In [72]:
url='https://www.quandl.com/api/v3/datasets/FSE/AFX_X.json?&start_date=2017-01-01&end_date=2017-12-31api_key=API_KEY'
r = requests.get(url)
yearly_data =r.json()

In [73]:
type(yearly_data)

dict

# 3 - Maximum and Minimum Opening Prices for the period 2017
By Analyzing the data we find that the column names (stock prices, volumes) etc are contained in the json_data under the list "column_names" while the actual values is itself under the key "data". Now that we have extracted the annual data for 2017, we need to parse the dict for key, value "data" and get the second value from this list - the value for opening price.

In [74]:
type(yearly_data['dataset']['data'])

list

Now we can iterate the above dictionary yearly_data and extract the value at index position (1)  and append these to another list named op_price

In [75]:
op_price=[]  # Create an empty list

for p in yearly_data['dataset']['data']:  # Iterate through the dictionary and get all the opening prices
    if p[1] != None:                      # Check if the opening price is a valid price
        op_price.append(p[1])             # Add the value to the array/list op_price so we can use the max/min methods
        
print ("Maximum Opening Price :", max(op_price))
print ("Minimum Opening Price :" , min(op_price))
    

Maximum Opening Price : 53.11
Minimum Opening Price : 34.0


# 4 -Maximum daily change in price based on Opening and Closing Price


In [76]:
daily_change = []  # Empty list to hold the daily change between High and Low prices
for p in yearly_data['dataset']['data']:  # Iterate through the dictionary and get the daily High(2) and Low (3) values
    if p[2] != None:
        high_price = p[2]
    if p[3] != None:
        low_price = p[3]
        
    dly_change = high_price - low_price
    daily_change.append(dly_change)

In [77]:
print ("Maximum Daily Change in Price :", max(daily_change))

Maximum Daily Change in Price : 2.8100000000000023


# 5 -Largest Change (intra-day) based on Closing Prices


In [78]:
closing_price =[]
for p in yearly_data['dataset']['data']:  # Iterate through the dictionary and get all the Closing prices
    if p[4] != None:                      # Check if the closing price is a valid price
        closing_price.append(p[4])  
        
max_closing_change = []    

for x in range (0,len(closing_price)):
    if x+1 != len(closing_price):
        close_price = closing_price[x] - closing_price[x+1]
        max_closing_change.append(close_price)
        
print("Maximum Change in Closing Price :", round(max(max_closing_change),2))

Maximum Change in Closing Price : 1.72


# 6 - Average Daily Trading Volume for the period (column 7 from the list)
get the daily trading volume and then we can get the mean of this

In [79]:
trading_volume = []
for p in yearly_data['dataset']['data']:  # Iterate through the dictionary and get all the Closing prices
    if p[6] != None:                      # Check if the closing price is a valid price
        trading_volume.append(p[6])  
        
        
print("Average Trading Volume for the period is :", sum(trading_volume)/len(trading_volume))      

Average Trading Volume for the period is : 89124.33725490196


# 7 - Median  Trading Volume calculation.

In [80]:

def median(lst):
    lst_sorted = sorted(lst)
    if len(lst)%2 == 1:
        med = lst_sorted[int(len(lst)/2)]
    if len(lst)%2 == 0:
        med = (lst_sorted[len(lst)//2] + lst_sorted[len(lst)//2 +1])/2
    return med

print('The median trading volume during 2017 = ', median(trading_volume))


# Define a function to calculate the Median. Median value is based on the length. If the length of the array is even
# median value is at the midpoint of the array. If the length is even, then median value is between the mid point and
# mid point -1. The length of the trading_volume is 255. So the median is 255/2 = volume@(127 + 128)/2


The median trading volume during 2017 =  76286.0
